In [44]:
# Make SQL
import sqlite3

conn = sqlite3.connect('valence_words.db')
c = conn.cursor()


c.execute("""CREATE TABLE IF NOT EXISTS words (
            english text UNIQUE,  
            danish text,
            syllables int,
            pleasure float,
            arousal float,
            dominance float
            )""")


conn.commit()
conn.close()



In [45]:
import deepl
auth_key = "78aa23d6-06f3-4228-880c-e07a38e5cebc:fx"  
deepl_client = deepl.DeepLClient(auth_key)

def translate(original):
    try:
        result = deepl_client.translate_text(original, target_lang="DA")
        return str(result)

    except Exception as e:
        print(f"An error occurred with Deepl: {e}")
        return None 



def count_syllables(word):
    vowels = 'aeiouyæøå'
    word = word.lower()
    syllable_count = 0
    in_vowel_group = False

    
    for char in word:
        if char in vowels:
            if not in_vowel_group:
                
                syllable_count += 1
                in_vowel_group = True
        else:
            in_vowel_group = False

    return syllable_count

In [34]:
print((translate('hello my name is hello')))

hej mit navn er hej


In [48]:

# WRITE TRANSLATIONS
import sqlite3


conn = sqlite3.connect('valence_words.db')
c = conn.cursor()

with open('word_data_officel.txt','r') as f_in:
    for n, line in enumerate(f_in):
        if n == 0:  
            continue
        
        # if n > 20: 
        #     break
        

        word, pleasure_str, arousal_str, dominance_str = line.strip().split(';')
        dansk_ord = translate(word).lower()
        print(word, dansk_ord)

        if dansk_ord: 
                pleasure = float(pleasure_str)
                arousal = float(arousal_str)
                dominance = float(dominance_str)
                syllables = count_syllables(dansk_ord)
                data_to_insert = (word, dansk_ord, syllables, pleasure, arousal, dominance)
                c.execute("INSERT INTO words VALUES (?,?,?,?,?,?)", data_to_insert)


conn.commit()
conn.close()


abduction bortførelse
abortion abort
absurd absurd
abundance overflod
abuse misbrug
acceptance accept
accident ulykke
ace ace
ache smerte
achievement præstation
activate aktivere
addict misbruger
addicted afhængig
admired beundret
adorable bedårende
adult voksen
advantage fordel
adventure eventyr
affection hengivenhed
afraid bange
aggressive aggressiv
agility smidighed
agony smerte
agreement aftale
air luft
alcoholic alkoholiker
alert alarm
alien udlænding
alimony underholdsbidrag
alive i live
allergy allergi
alley gyde
alone alene
aloof reserveret
ambition ambition
ambulance ambulance
angel engel
anger anger
angry vred
anguished forpint
ankle ankel
annoy irritere
answer svar
anxious ængstelig
applause bifald
appliance apparat
arm arm
army hær
aroused ophidset
arrogant arrogant
art kunst
assassin lejemorder
assault overfald
astonished forbløffet
astronaut astronaut
athletics atletik
autumn efterår
avalanche lavine
avenue avenue
awed forbløffet
baby baby
bake bage
bandage bandage
bankru

In [49]:
conn = sqlite3.connect('valence_words.db')
c = conn.cursor()


c = conn.execute('SELECT * FROM words')
rows = c.fetchall()
print(len(rows))



conn.commit()
conn.close()

1034


In [3]:
import sqlite3
import csv


conn = sqlite3.connect('valence_words.db')
c = conn.cursor()


c = conn.execute("""SELECT english, danish, pleasure, arousal, dominance FROM words
                    WHERE syllables = 2
                    ORDER BY pleasure ASC
                    LIMIT 150;
                    """)

rows = c.fetchall()
used_danish_words = set()

with open('negative_ord.csv', 'a', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['English', 'Danish', 'Pleasure', 'Arousal', 'Dominance'])  
    
    for row in rows:
        danish_word = row[1]
        if danish_word not in used_danish_words:
            used_danish_words.add(danish_word)
            writer.writerow(row)  


conn.commit()
conn.close()

In [5]:
import sqlite3
conn = sqlite3.connect('valence_words.db')
c = conn.cursor()


sql_query = """
    SELECT english, danish, pleasure, arousal, dominance
    FROM words
    WHERE
        syllables = 2
        AND arousal < 59.190326503667485 * 0.8
    ORDER BY
        ABS(pleasure - (SELECT AVG(pleasure) FROM words WHERE syllables = 2)) ASC
    LIMIT 100;
"""


c = conn.execute(sql_query)
rows = c.fetchall()
used_danish_words = set()

with open('neutrale_ord.csv', 'a', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['English', 'Danish', 'Pleasure', 'Arousal', 'Dominance'])  
    
    for row in rows:
        danish_word = row[1]
        if danish_word not in used_danish_words:
            used_danish_words.add(danish_word)
            writer.writerow(row)  

conn.commit()
conn.close()

In [ ]:
# AVG pleasure
import sqlite3
conn = sqlite3.connect('valence_words.db')
c = conn.cursor()

# Ask SQL to calculate the average pleasure directly
sql_query = """
    SELECT AVG(pleasure) FROM words
    WHERE syllables = 2
"""

c.execute(sql_query)

# fetchone() gets the single result row
average_pleasure = c.fetchone()[0]

print(f"The average pleasure is: {average_pleasure}")

conn.close()

The average pleasure is: 59.190326503667485


In [ ]:
import sqlite3
conn = sqlite3.connect('valence_words.db')
c = conn.cursor()


sql_query = """
    SELECT english, danish, pleasure, arousal, dominance
    FROM words
    ORDER BY
        dominance ASC
    LIMIT 20;
"""


c = conn.execute(sql_query)
rows = c.fetchall()

for row in rows:
    print(row)
conn.commit()
conn.close()

In [23]:
import os

files = [file for file in os.listdir('negative-ord-real') if file.endswith('mp3')]

def get_num(file_name):
    return int(file_name.split()[1].split('.')[0])

files.sort(key = get_num)

with open('negative_ord.csv', 'r') as f:
    names = []
    for i, line in enumerate(f):
        if i == 0:
            continue
        try:
            names.append(line.split(',')[1])
        
        except:
            break

for i, file in enumerate(files):
    old_path = os.path.join('negative-ord-real', file)  # Correct the old path
    new_name = f"{names[i].replace(' ', '_')}.mp3"  # Replace spaces with underscores in new names
    new_path = os.path.join('negative-ord-real', new_name)

    # Debug: Print paths
    print("Old Path:", os.path.abspath(old_path))
    print("New Path:", os.path.abspath(new_path))

    # Check if the old file exists before renaming
    if os.path.exists(old_path):
        os.rename(old_path, new_path)
    else:
        print(f"File not found: {old_path}")


Old Path: /Users/johanandersen/Desktop/EM2_Kode/auditory-priming/words-for-EM/negative-ord-real/Sound 73.mp3
New Path: /Users/johanandersen/Desktop/EM2_Kode/auditory-priming/words-for-EM/negative-ord-real/voldtægt.mp3
Old Path: /Users/johanandersen/Desktop/EM2_Kode/auditory-priming/words-for-EM/negative-ord-real/Sound 75.mp3
New Path: /Users/johanandersen/Desktop/EM2_Kode/auditory-priming/words-for-EM/negative-ord-real/selvmord.mp3
Old Path: /Users/johanandersen/Desktop/EM2_Kode/auditory-priming/words-for-EM/negative-ord-real/Sound 76.mp3
New Path: /Users/johanandersen/Desktop/EM2_Kode/auditory-priming/words-for-EM/negative-ord-real/afvist.mp3
Old Path: /Users/johanandersen/Desktop/EM2_Kode/auditory-priming/words-for-EM/negative-ord-real/Sound 77.mp3
New Path: /Users/johanandersen/Desktop/EM2_Kode/auditory-priming/words-for-EM/negative-ord-real/morder.mp3
Old Path: /Users/johanandersen/Desktop/EM2_Kode/auditory-priming/words-for-EM/negative-ord-real/Sound 79.mp3
New Path: /Users/johana